In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pandas import DataFrame, Index, Series
from torch.utils.data import Dataset as TorchDataset

import tsdm
from tsdm.datasets import TimeSeriesCollection
from tsdm.random.samplers import HierarchicalSampler, SlidingWindowSampler

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
ds = tsdm.datasets.KIWI_RUNS()

In [ ]:
ds.time_features.dtypes

In [ ]:
TSC = TimeSeriesCollection(
    index=ds.index,
    timeseries=ds.timeseries,
    metadata=ds.metadata,
    time_features=ds.time_features,
    value_features=ds.value_features,
    metadata_features=ds.metadata_features,
)

## Sampler

In [ ]:
ts = ds.timeseries.loc[(439, 15325)]

In [ ]:
sampler = SlidingWindowSampler(ts.index, horizons=["2h", "1h"], stride="1h")

In [ ]:
next(iter(sampler))

## Construct the Sampler

In [ ]:
TSC

In [ ]:
from torch.utils.data import RandomSampler, SubsetRandomSampler

In [ ]:
next(iter(SubsetRandomSampler(TSC.index)))

In [ ]:
TSC[(439, 15325)]

In [ ]:
subsamplers = {
    key: SlidingWindowSampler(ds.timeseries.index, horizons=["2h", "1h"], stride="1h")
    for key, ds in TSC.items()
}
sampler = HierarchicalSampler(TSC, subsamplers, shuffle=False)

In [ ]:
outer_key, (forecasting_horizon, prediction_horizon) = next(iter(sampler))

In [ ]:
TSC[outer_key][forecasting_horizon]

# TimeSeriesCollectionForecastingTask

In [ ]:
TSC

In [ ]:
from tsdm.tasks import TimeSeriesCollectionTask

targets = ["Base", "DOT", "Glucose", "OD600"]
observables = [
    "Base",
    "DOT",
    "Glucose",
    "OD600",
    "Acetate",
    "Fluo_GFP",
    "pH",
]
covariates = [
    "Cumulated_feed_volume_glucose",
    "Cumulated_feed_volume_medium",
    "InducerConcentration",
    "StirringSpeed",
    "Flow_Air",
    "Temperature",
    "Probe_Volume",
]

In [ ]:
key = next(iter(sampler))
outer_key, (observation_horizon, forecasting_horizon) = key
TSC[outer_key][observation_horizon]

In [ ]:
task = TimeSeriesCollectionTask(
    TSC,
    targets=targets,
    observables=observables,
    covariates=covariates,
    sample_format=("sparse", "sparse"),
)
sample = task[key]

In [ ]:
if sample.inputs.t_target is not None:
    diff = sample.inputs.t_target.index.difference(sample.targets.y.index)
    sample.inputs.t_target.drop(diff, inplace=True)
sample

In [ ]:
raise

In [ ]:
diff = sample.inputs.t_target.index.difference(sample.targets.y.index)
sample.inputs.t_target.drop(diff, inplace=True)
sample

In [ ]:
raise

# Mapping Dataset

In [ ]:
d = dict(enumerate("asdfghjkl"))

In [ ]:
from collections.abc import Mapping
from dataclasses import dataclass

from torch.utils.data import Dataset

In [ ]:
@dataclass
class MyMapping(Dataset, Mapping):
    internal_dict: dict

    def __iter__(self):
        return iter(self.internal_dict)

    def __getitem__(self, key):
        return self.internal_dict[key]

    def __len__(self):
        return len(self.internal_dict)

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(MyMapping(d))